# 基本操作

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.8.0
False


# 自动求导

In [7]:
from torch import autograd
x = torch.tensor(7.)  # 默认requires_grad是False
a = torch.tensor(999., requires_grad=True)
b = torch.tensor(20., requires_grad=True)
c = torch.tensor(30., requires_grad=True)
y = a ** 2 * x + b * x + c
grads = autograd.grad(y, [a, b, c])  # y函数对a,b,c求导
print(grads[0], grads[1], grads[2])
print(grads[0].item() == 7 * 999 * 2)
print(grads[1].item() == 7)
print(grads[2].item() == 1)

tensor(13986.) tensor(7.) tensor(1.)
True
True
True


# mnist

In [163]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch import optim

In [165]:
def loadData(filename):
    x = []
    y = []
    with open(filename, 'r')as f:
        for line in f.readlines():
            cur_line = line.strip().split(',')
            x.append([int(i) / 255 for i in cur_line[1:]])  # 归一化,不归一化训练时会出现loss=nan的情况
            y.append(int(cur_line[0]))
    return x, y

def normalize(x):  # 归一化
    for i in range(len(x)):
        x[i] = [(j - min(x[i])) / (max(x[i]) - min(x[i])) for j in x[i]]

class mnist_dataset(Dataset):
    def __init__(self, x, y):
        self.len = len(x)
        self.x = np.array(x, dtype=np.float32)
        self.y = np.array(y, dtype=np.float32)
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)
    
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.x[index], self.y[index]

train_x , train_y = loadData('mnist/mnist_train.csv')
test_x, test_y = loadData('mnist/mnist_test.csv')

train_data = DataLoader(dataset=mnist_dataset(train_x, train_y), batch_size=512, shuffle=True)
test_data = DataLoader(dataset=mnist_dataset(test_x, test_y), batch_size=512, shuffle=True)
x_tmp, y_tmp = next(iter(train_data))
print(f'数据shape:{x_tmp.shape}')
print(f'数据集类型{x_tmp.dtype}')
print(torch.min(x_tmp))
print(torch.max(x_tmp))
print(torch.max(y_tmp))
print(torch.min(y_tmp))

数据shape:torch.Size([512, 784])
数据集类型torch.float32
tensor(0.)
tensor(1.)
tensor(9.)
tensor(0.)


In [166]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [167]:
def one_hot(label):
    label = label.long()
    return torch.zeros(len(label), 10).scatter_(1, label.view(-1, 1), 1)
label = torch.tensor([1., 2., 3., 5., 9.], dtype=torch.float32)
print(one_hot(label))

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [168]:
for epoch in range(20):
    for batch_idx, (x, y) in enumerate(train_data):
        out = net(x)
        loss = F.mse_loss(out, one_hot(y))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print(f'epoch[{epoch + 1}/{20}]:[{batch_idx}/{len(train_data)}]  loss={loss.item()}')

epoch[1/20]:[0/118]  loss=0.10734939575195312
epoch[1/20]:[50/118]  loss=0.08664204925298691
epoch[1/20]:[100/118]  loss=0.08042768388986588
epoch[2/20]:[0/118]  loss=0.07889939844608307
epoch[2/20]:[50/118]  loss=0.07275237888097763
epoch[2/20]:[100/118]  loss=0.06787366420030594
epoch[3/20]:[0/118]  loss=0.06601966917514801
epoch[3/20]:[50/118]  loss=0.06129629164934158
epoch[3/20]:[100/118]  loss=0.05830339714884758
epoch[4/20]:[0/118]  loss=0.056342411786317825
epoch[4/20]:[50/118]  loss=0.05209506303071976
epoch[4/20]:[100/118]  loss=0.05069391801953316
epoch[5/20]:[0/118]  loss=0.049439746886491776
epoch[5/20]:[50/118]  loss=0.047182101756334305
epoch[5/20]:[100/118]  loss=0.04756007343530655
epoch[6/20]:[0/118]  loss=0.04524365812540054
epoch[6/20]:[50/118]  loss=0.04342168942093849
epoch[6/20]:[100/118]  loss=0.042544059455394745
epoch[7/20]:[0/118]  loss=0.04335609823465347
epoch[7/20]:[50/118]  loss=0.041015900671482086
epoch[7/20]:[100/118]  loss=0.040989089757204056
epoch[8

In [169]:
total = 0
correct_num = 0
with torch.no_grad():
    for x, y in test_data:
        total += len(x)
        out = net(x)
        out = torch.argmax(out, dim=1)
        correct_num += out.eq(y).sum().float()
print(f'acc={correct_num / total}')

acc=0.9136999845504761
